### Simulation datasets

This notebook creates the datasets for the simulation study according to the different DGPs that are explained in the paper. 
Note that a slight change in the parameters might lead to either an implosive or explosive process. Be careful with tuning these parameters.

In [1]:
%matplotlib inline

import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal
from scipy import io

# Own code
sys.path.append("../")
from utils.data_utils import create_data, create_dgp_data, transformation, standardize, generate_dgp_tvp_var, generate_dgp_tvp_var_heteroskedastic, generate_matrices, generate_contemp_matrices
from utils.tvp_models import TVPVARModel, tvp_ar_contemp, tvp_ar_non_contemp

# Suppress scientific notation in numpy
np.set_printoptions(suppress=True)

In [ ]:
different_m = [3,7]
different_T = [100,200]
different_sparsity = [0.20,0.40]
explosive_threshold = 25

for M in different_m:
    for T in different_T:
        for p_0 in different_sparsity:

            np.random.seed(12345)
            n_iterations = 500

            p = 1
            
            counter = 0

            for run in range(n_iterations):
                
                y_heteroskedastic, coefficients_heteroskedastic, *_ = generate_dgp_tvp_var_heteroskedastic(M, T, p, 1/3, 1/9, 4*1e-5, 1/6, 1/2, p_0, 1e-2, 1e-9)

                x_heteroskedastic = y_heteroskedastic[:,:-1]
                np.savetxt(f'../simulations/datasets/y_{M}_{T}_{p}_{p_0}_{run+1}_het.csv',y_heteroskedastic[:,1:].T, delimiter=",")
                np.savetxt(f'../simulations/datasets/x_{M}_{T}_{p}_{p_0}_{run+1}_het.csv',x_heteroskedastic.T, delimiter=",")
                np.savetxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{p_0}_{run+1}_het.csv', coefficients_heteroskedastic[:,1:], delimiter=",")

                np.savetxt(f'../simulations/datasets/y_{M}_{T}_{p}_{p_0}_{run+1}_het_py.csv',y_heteroskedastic, delimiter=",")
                np.savetxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{p_0}_{run+1}_het_py.csv', coefficients_heteroskedastic, delimiter=",")
                
                y_max = y_heteroskedastic.max()
                y_min = y_heteroskedastic.min()
                if (y_max > explosive_threshold or y_min < -explosive_threshold):
                    print(f"y is an explosive process -> RUN: {run} | MAX: {y_max} & MIN: {y_min}")
                    counter += 1
                
            print(f"M: {M}, T: {T}, p_0: {p_0} & explosive: {round((counter/n_iterations)*100,6)} -> DONE!")